In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [6]:
import os

# Defining paths

In [7]:
train_path = "/Users/andrei/code/images/images/train"

In [8]:
test_path = "/Users/andrei/code/images/images/validation"

In [9]:
val_path = "~/code/images/images/val"

# Loading the images

In [10]:
# iterate through the names of contents of the folder
all_images = []
y_train = []
X_train = []
for folder_path in os.listdir(train_path):
    if not folder_path.startswith("."):
        for image_path in os.listdir(os.path.join(train_path, folder_path)):
            img = load_img(os.path.join(os.path.join(train_path, folder_path), image_path), color_mode = "grayscale")
            X_train.append(img_to_array(img))
            y_train.append(os.path.basename(os.path.normpath(folder_path)))

In [11]:
y_train = np.array(y_train)

In [12]:
X_train = np.array(X_train)

## Test


In [9]:
# iterate through the names of contents of the folder
all_images = []
y_test = []
X_test = []

for folder_path in os.listdir(test_path):
    if not folder_path.startswith("."):
        for image_path in os.listdir(os.path.join(test_path, folder_path)):
            img = load_img(os.path.join(os.path.join(test_path, folder_path), image_path), color_mode = "grayscale")
            X_test.append(img_to_array(img))
            y_test.append(os.path.basename(os.path.normpath(folder_path)))

In [10]:
y_test = np.array(y_test)

In [11]:
X_test = np.array(X_test)

# Investigation and baseline

In [12]:
y_train_df = pd.DataFrame(y_train)

In [13]:
y_test_df = pd.DataFrame(y_test)

In [14]:
y_train_df.value_counts(normalize=True)

happy       0.248569
neutral     0.172860
sad         0.171333
fear        0.142361
angry       0.138545
surprise    0.111204
disgust     0.015128
dtype: float64

In [15]:
y_test_df.value_counts(normalize=True)

happy       0.258279
neutral     0.172092
sad         0.161194
fear        0.144070
angry       0.135862
surprise    0.112794
disgust     0.015709
dtype: float64

In [16]:
baseline = y_train_df.value_counts(normalize=True).max()

In [17]:
baseline

0.2485687519517019

In [18]:
X_train.shape

(28821, 48, 48, 1)

In [19]:
X_test.shape

(7066, 48, 48, 1)

# Preprocessing

In [39]:
from sklearn.preprocessing import OneHotEncoder

In [40]:
ohe = OneHotEncoder(sparse = False)

In [41]:
y_train_reshaped = y_train.reshape(-1, 1)

In [42]:
y_test_reshaped = y_test.reshape(-1, 1)

NameError: name 'y_test' is not defined

In [43]:
y_train_cat = ohe.fit_transform(y_train_reshaped)

In [ ]:
y_test_cat = ohe.transform(y_test_reshaped)

In [26]:
ohe.get_feature_names_out()

array(['x0_angry', 'x0_disgust', 'x0_fear', 'x0_happy', 'x0_neutral',
       'x0_sad', 'x0_surprise'], dtype=object)

In [44]:
y_train_cat.shape

(28821, 7)

In [28]:
X_train.shape

(28821, 48, 48, 1)

In [50]:
from sklearn.utils import shuffle

In [30]:
X_shuffle, y_shuffle = shuffle(X_train, y_train_cat, random_state = 0)

In [31]:
X_sample = X_shuffle[:3000, :, :, :]

In [32]:
y_sample = y_shuffle[:3000]

## Removing disgust

In [45]:
y_train_bal = y_train_cat[:28385, :]

In [47]:
y_train_bal.shape

(28385, 7)

In [46]:
X_train_bal = X_train[:28385, :, :, :]

In [48]:
X_train_bal.shape

(28385, 48, 48, 1)

# Model

In [54]:
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import layers, models

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import layers, models

model = models.Sequential()

# Notice this cool new layer that "pipe" your rescaling within the architecture
model.add(Rescaling(1./255, input_shape=(48, 48, 1)))

# Lets add 3 convolution layers, with relatively large kernel size as our pictures are quite big too
model.add(layers.Conv2D(16, kernel_size=5, activation='relu'))
model.add(layers.MaxPooling2D(3))

model.add(layers.Conv2D(32, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(3))

model.add(layers.Conv2D(32, kernel_size=2, activation="relu"))
model.add(layers.MaxPooling2D(3))

model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [65]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model.fit(X_sample, y_sample, batch_size = 32, epochs = 1000,
          callbacks=[EarlyStopping(patience = 20, restore_best_weights= True, monitor = "val_accuracy", mode = "max")],
         validation_split = 0.2, verbose = 1)

In [ ]:
model.evaluate(X_test, y_test_cat)

# Fitting on all of the data

In [51]:
X_shuffle_all, y_shuffle_all = shuffle(X_train_bal, y_train_bal, random_state = 0)

In [82]:
model2 = models.Sequential()

# Notice this cool new layer that "pipe" your rescaling within the architecture
model2.add(Rescaling(1./255, input_shape=(48, 48, 1)))

# Lets add 3 convolution layers, with relatively large kernel size as our pictures are quite big too
model2.add(layers.Conv2D(50, kernel_size=3, activation='relu'))
model2.add(layers.Conv2D(50, kernel_size=3, activation='relu'))
model2.add(layers.Dropout(.4))
model2.add(layers.MaxPooling2D(2))

model2.add(layers.Conv2D(30, kernel_size=3, activation="relu"))
model2.add(layers.Conv2D(30, kernel_size=3, activation="relu"))
model2.add(layers.Dropout(.4))
model2.add(layers.MaxPooling2D(2))

model2.add(layers.Conv2D(20, kernel_size=1, activation="relu"))
model2.add(layers.Conv2D(20, kernel_size=1, activation="relu"))
model2.add(layers.Dropout(.4))

model2.add(layers.Flatten())
model2.add(layers.Dense(100, activation='relu'))
model2.add(layers.Dense(7, activation='softmax'))

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [83]:
model2.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_13 (Rescaling)    (None, 48, 48, 1)         0         
                                                                 
 conv2d_55 (Conv2D)          (None, 46, 46, 50)        500       
                                                                 
 conv2d_56 (Conv2D)          (None, 44, 44, 50)        22550     
                                                                 
 dropout_6 (Dropout)         (None, 44, 44, 50)        0         
                                                                 
 max_pooling2d_39 (MaxPoolin  (None, 22, 22, 50)       0         
 g2D)                                                            
                                                                 
 conv2d_57 (Conv2D)          (None, 20, 20, 30)        13530     
                                                     

In [ ]:
model2.fit(X_shuffle_all, y_shuffle_all, batch_size = 32, epochs = 1000,
          callbacks=[EarlyStopping(patience = 10, restore_best_weights= True, monitor = "val_accuracy", mode = "max")],
         validation_split = 0.2, verbose = 1)

Epoch 1/1000
710/710 [==============================] - 105s 147ms/step - loss: 1.6821 - accuracy: 0.3000 - val_loss: 1.5851 - val_accuracy: 0.3622
Epoch 2/1000
710/710 [==============================] - 100s 141ms/step - loss: 1.5020 - accuracy: 0.4002 - val_loss: 1.4892 - val_accuracy: 0.4212
Epoch 3/1000
710/710 [==============================] - 100s 141ms/step - loss: 1.4194 - accuracy: 0.4352 - val_loss: 1.3899 - val_accuracy: 0.4531
Epoch 4/1000
710/710 [==============================] - 108s 153ms/step - loss: 1.3385 - accuracy: 0.4707 - val_loss: 1.3333 - val_accuracy: 0.4842
Epoch 5/1000
710/710 [==============================] - 106s 149ms/step - loss: 1.2822 - accuracy: 0.4916 - val_loss: 1.2882 - val_accuracy: 0.4934
Epoch 6/1000
710/710 [==============================] - 106s 149ms/step - loss: 1.2361 - accuracy: 0.5131 - val_loss: 1.2507 - val_accuracy: 0.5158
Epoch 7/1000
710/710 [==============================] - 100s 141ms/step - loss: 1.2070 - accuracy: 0.5226 - val_

In [ ]:
model2.evaluate(X_test, y_test_cat)

In [ ]:
y_pred = model2.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
output = ConfusionMatrixDisplay(confusion_matrix(np.argmax(y_test_cat, axis =1), np.argmax(y_pred, axis =1)))

In [ ]:
output.plot()

In [ ]:
y_test

In [ ]:
ohe.get_feature_names_out()

In [ ]:
y_test

# Uploading a temp model to mlflow

In [41]:
import mlflow

In [46]:
import time

In [49]:
def save_model(model: Model = None,
               params: dict = None,
               metrics: dict = None) -> None:
    """
    persist trained model, params and metrics
    """

    timestamp = time.strftime("%Y%m%d-%H%M%S")

    # retrieve mlflow env params
    mlflow_tracking_uri = "https://mlflow.lewagon.ai"
    mlflow_experiment = "[batch-960]-job_prepr_temp"
    mlflow_model_name = "[batch-960]-job_prepr_temp"

    # configure mlflow
    mlflow.set_tracking_uri(mlflow_tracking_uri)
    mlflow.set_experiment(experiment_name=mlflow_experiment)

    with mlflow.start_run():

        mlflow.keras.log_model(keras_model=model,
                                   artifact_path="model",
                                   keras_module="tensorflow.keras",
                                   registered_model_name=mlflow_model_name)

    print("\n✅ data saved to mlflow")

    return None

In [50]:
save_model(model2)

2022/09/06 11:55:39 INFO mlflow.tracking.fluent: Experiment with name '[batch-960]-job_prepr_temp' does not exist. Creating a new experiment.


INFO:tensorflow:Assets written to: /var/folders/lg/x7r61bz54jlf6d0nvwrlqd0m0000gn/T/tmp8whg0dcx/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/lg/x7r61bz54jlf6d0nvwrlqd0m0000gn/T/tmp8whg0dcx/model/data/model/assets
Successfully registered model '[batch-960]-job_prepr_temp'.
2022/09/06 11:55:53 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: [batch-960]-job_prepr_temp, version 1



✅ data saved to mlflow


Created version '1' of model '[batch-960]-job_prepr_temp'.


# Load model from mlflow

In [1]:
import mlflow

In [2]:
mlflow.set_tracking_uri('https://mlflow.lewagon.ai/')
mlflow_model_name = '[batch-960]-job_prepr_temp'
model_uri = f"models:/{mlflow_model_name}/Production"
try:
    model = mlflow.keras.load_model(model_uri=model_uri)
except:
    raise Exception(f'No model in Production on mlflow')

2022-09-06 14:30:32.581102: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
model.get_config()

{'name': 'sequential',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 48, 48, 1),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'rescaling_input'}},
  {'class_name': 'Rescaling',
   'config': {'name': 'rescaling',
    'trainable': True,
    'batch_input_shape': (None, 48, 48, 1),
    'dtype': 'float32',
    'scale': 0.00392156862745098,
    'offset': 0.0}},
  {'class_name': 'Conv2D',
   'config': {'name': 'conv2d',
    'trainable': True,
    'dtype': 'float32',
    'filters': 16,
    'kernel_size': (5, 5),
    'strides': (1, 1),
    'padding': 'valid',
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'groups': 1,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': 

In [4]:
dir(model)

['_SCALAR_UPRANKING_ON',
 '_TF_MODULE_IGNORED_PROPERTIES',
 '__call__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_trackable',
 '_add_trackable_child',
 '_add_variable_with_custom_getter',
 '_assert_compile_was_called',
 '_assert_weights_created',
 '_auto_track_sub_layers',
 '_autocast',
 '_autographed_call',
 '_base_model_initialized',
 '_build_graph_network_for_inferred_shape',
 '_build_input_shape',
 '_call_accepts_kwargs',
 '_call_arg_was_passed',
 '_call_fn_arg_defaults',
 '_call_fn_arg_positions',
 '_call_fn_args',
 '_call_full_argspec',
 '_callable_losses',
 '_ca